# Zähme die Tabellen des Grauens

Lächle und sei froh, es könnte noch schlimmer kommen. Und ich lächelte und war froh ... und es kam schlimmer.

Die Destatis-Flat-Tables sind im Vergleich zu anderen Datenportalen schon ganz gut.
Das geht geht auch schlimmer: 

Wir zeigen dir jetzt einmal das Datenportal [regionalstatistik.de](https://www.regionalstatistik.de/genesis/online/)

Ziel ist eine Choroplethen-Karte wie diese:

In [1]:
# Display the associated webpage in a new window
import IPython
IPython.display.IFrame('https://datawrapper.dwcdn.net/IJwst/4/', width='600', height='750')

### 1. Daten als .csv herunterladen

Suche auf regionalstatistik.de nach ‘Bauland’ oder direkt nach der Nummer der Tabelle: 61511-01-03-4. Lade die Daten als .csv herunter und speichere sie im ./data Ordner ab.

### 2. Tabelle anschauen

Der erste Fehler fällt dir schon auf, wenn du versuchst, die Tabelle im Jupyter Notebook anzuklicken. 

Du bekommst folgende Fehlermeldung:

```"File Load Error for 61511-01-03-4.csv
/home/p3nny/pfv/data/61511-01-03-4.csv is not UTF-8 encoded"
```

Das heißt, der Zeichensatz, mit dem das Dokument angelegt ist, entspricht nicht dem Standard utf-8 sondern es wird ein deutsches Sonderformat verwendet.

Öffne die .csv bitte mal in Excel oder Google Spreadsheets etc.

Vor Dir siehst du eine recht verschachtelte Tabelle. Sie ist gut geeignet, um auf DIN-A4-Seiten ausgedruckt zu werden, Pandas oder ein Visualisierungstool wie Datawrapper ist damit jedoch überfordert.

### 3. In pandas einlesen

Versuche mal die Tabelle per read_csv in Pandas einzulesen. Denke daran, dass du pandas erst einmal importieren musst.

In [2]:
import pandas as pd 

In [3]:
df = pd.read_csv("./data/61511-01-03-4.csv")

ParserError: Error tokenizing data. C error: Expected 4 fields in line 27, saw 5





Das wird nicht einfach, aber es geht. Allerdings musst du pandas an vielen Stellen auf die Sprünge helfen.

In [4]:
df = pd.read_csv('./data/61511-01-03-4.csv', sep=';', encoding='ISO-8859-1', skiprows=7, 
                 skipfooter=4, engine="python",
                 decimal=',',
                names=['Jahr', 'AGS', 'Ort', 'Art', 'Einheit', 'Insgesamt', 'baureifes Land'],
                dtype={'AGS': 'str'})

Hier kommt die Schritt für Schritt Erklärung:
    
sep=';' 
In Deutschland wird anders als in den USA nicht das Komma als Trenner verwendet (das ist ja bereits belegt, weil wir '3,3' statt '3.3' schreiben), sondern der Trenner ist ein Semikolon ';'.

encoding='ISO-8859-1'
Setzt das zeichenformat auf 'ISO-8859-1', eine andere oft von deutschen Behörden genutzte Variante ist 'latin-1'. Welches verwendet wurde, weiß man vorher nicht. Deshalb müsst ihr die durchprobieren, falls utf-8 nicht geht. 

skiprows=7
Überspringt den verschachtelten Tabellenkopf. Wie viele Zeilen du überspringen musst, ist von .csv zu .csv unterschiedlich. Auch hier hilft nur ausprobieren.

skipfooter=4, engine="python"
Skipfooter lässt Zeilen unten weg, die nicht zur Tabelle gehören. Diese Funktion erfordert die Angabe engine="python", sonst gibt es eine Fehlermeldung.

decimal=','
Gibt an, dass Python ',' als '.' bei Kommazahlen interpretieren soll: '3.2' statt '3.3'

Damit lässt sich die Tabelle bereits einlesen. Hilfreich sind diese noch diese beiden Parameter:

names=['Jahr', 'AGS', 'Ort', 'Art', 'Einheit', 'Insgesamt', 'baureifes Land']
Gibt an, wie die Spalten heißen sollen

dtype={'AGS': 'str'}
Gibt schon beim einlesen an, dass der AGS als String eingelesen werden soll. Wenn der als Zahl erkannt wird, dann geht die führende Null verloren.


In [5]:
df

,Jahr,AGS,Ort,Art,Einheit,Insgesamt,baureifes Land
0,2018,DG,Deutschland,Veräußerungsfälle von Bauland,Anzahl,84272,74692
1,2018,DG,Deutschland,Veräußerte Baulandfläche,1000 qm,130011,77147
2,2018,DG,Deutschland,Kaufsumme,Tsd. EUR,17898272,14574917
3,2018,DG,Deutschland,Durchschnittlicher Kaufwert je qm,EUR,137.67,188.92
4,2018,01,Schleswig-Holstein,Veräußerungsfälle von Bauland,Anzahl,2003,1760
...,...,...,...,...,...,...,...
2147,2018,16076,"Greiz, Kreis",Durchschnittlicher Kaufwert je qm,EUR,20.88,25.26
2148,2018,16077,"Altenburger Land, Kreis",Veräußerungsfälle von Bauland,Anzahl,121,111
2149,2018,16077,"Altenburger Land, Kreis",Veräußerte Baulandfläche,1000 qm,179,75
2150,2018,16077,"Altenburger Land, Kreis",Kaufsumme,Tsd. EUR,3059,1814


### 3. Entscheiden, was visualisiert werden soll

Um eine Flächenwert-Stufen-Karte (Choroplethenkarte) zu erstellen, musst du zwei Entscheidungen treffen:

- Welche regionale Ebene möchtest du darstellen? In diesem Fall möchten wir Kreise in Nordrhein-Westfalen zeigen.
- Nach welchem Merkmal soll die Karte eingefärbt werden? Um Vergleichbarkeit herzustellen, möchten wir die Kreise anhand des durchschnittlichen Kaufwert je qm einfärben.

### 4. Filtern auf 'durchschnittlicher Kaufwert je qm' 

In [6]:
df = df[df['Art'] == 'Durchschnittlicher Kaufwert je qm']

In [7]:
df

,Jahr,AGS,Ort,Art,Einheit,Insgesamt,baureifes Land
3,2018,DG,Deutschland,Durchschnittlicher Kaufwert je qm,EUR,137.67,188.92
7,2018,01,Schleswig-Holstein,Durchschnittlicher Kaufwert je qm,EUR,85.30,120.26
11,2018,01001,"Flensburg, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,.,.
15,2018,01002,"Kiel, Landeshauptstadt, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,.,.
19,2018,01003,"Lübeck, Hansestadt, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,110.84,144.06
...,...,...,...,...,...,...,...
2135,2018,16073,"Saalfeld-Rudolstadt, Kreis",Durchschnittlicher Kaufwert je qm,EUR,42.50,41.80
2139,2018,16074,Saale-Holzland-Kreis,Durchschnittlicher Kaufwert je qm,EUR,44.25,49.51
2143,2018,16075,Saale-Orla-Kreis,Durchschnittlicher Kaufwert je qm,EUR,24.05,25.66
2147,2018,16076,"Greiz, Kreis",Durchschnittlicher Kaufwert je qm,EUR,20.88,25.26


### 5. Filtern nach dem Allgemeinen Gemeindeschlüssel, AGS

Der amtliche Gemeindeschlüssel (AGS) ist eigentlich eine achtstellige Zahl, die alle Städte und politisch eigenständigen Gemeinden in Deutschland eindeutig identifiziert.
In unserer Tabelle haben wir Daten bis runter auf Kreisebene, deshalb sind die ersten fünf Stellen angegeben.
Wir interessieren uns nur für NRW, dessen Gemeindeschlüssel beginnen immer mit 05.

[Hier wird der AGS interaktiv erklärt](https://datengui.de/statistik-erklaert/ags)

In [8]:
df = df[df.AGS.str.startswith('05')]

In [9]:
df

,Jahr,AGS,Ort,Art,Einheit,Insgesamt,baureifes Land
299,2018,05,Nordrhein-Westfalen,Durchschnittlicher Kaufwert je qm,EUR,99.49,137.71
303,2018,051,"Düsseldorf, Regierungsbezirk",Durchschnittlicher Kaufwert je qm,EUR,101.67,191.65
307,2018,05111,"Düsseldorf, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,959.98,959.98
311,2018,05112,"Duisburg, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,254.82,277.14
315,2018,05113,"Essen, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,274.93,322.64
...,...,...,...,...,...,...,...
523,2018,05962,Märkischer Kreis,Durchschnittlicher Kaufwert je qm,EUR,47.72,47.79
527,2018,05966,"Olpe, Kreis",Durchschnittlicher Kaufwert je qm,EUR,91.93,91.93
531,2018,05970,"Siegen-Wittgenstein, Kreis",Durchschnittlicher Kaufwert je qm,EUR,91.23,94.47
535,2018,05974,"Soest, Kreis",Durchschnittlicher Kaufwert je qm,EUR,74.75,106.72


Leider haben wir in der Tabelle gemischte Werte für das gesamte Bundesland (AGS: 05), für die Regierungsbezirke (AGS zum Beispiel: 051) und für die Kreise (AGS zum Beispiel: 05111).
So können wir die Daten nicht in Datawrapper packen, der erwartet auf einer Kreis-Karte nur Landkreise. 
Deshalb brauchen wir einen Trick, um alle Zeilen rauszufiltern, bei denen der AGS weniger als 5 Stellen hat:

Ein Weg ist für jeden Wert in der Spalte AGS zu gucken, ob die Länge des Strings größer als 3 ist. Erinnert ihr euch an die len('penny) Funktion?

In [10]:
def check_if_larger_than_3(x):
    if len(x) > 3:
        return True
    else:
        return False

Diese Funktion müssen wir dann auf alle Werte in der Spalte AGS anwenden. Dort wird dann entweder 'False' oder 'True' ausgegeben.

In [11]:
df.AGS.apply(check_if_larger_than_3)

299    False
303    False
307     True
311     True
315     True
       ...  
523     True
527     True
531     True
535     True
539     True
Name: AGS, Length: 61, dtype: bool

Pandas kann eine solche Funktion nutzen, um den Dataframe zu filtern: 

In [12]:
df[df.AGS.apply(check_if_larger_than_3)].head()

,Jahr,AGS,Ort,Art,Einheit,Insgesamt,baureifes Land
307,2018,05111,"Düsseldorf, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,959.98,959.98
311,2018,05112,"Duisburg, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,254.82,277.14
315,2018,05113,"Essen, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,274.93,322.64
319,2018,05114,"Krefeld, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,52.63,146.35
323,2018,05116,"Mönchengladbach, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,85.40,128.48


Um Schreibarbeit zu sparen, nutzt man in der Datenanalyse einen kürzeren Weg, um die Funktion zu definiieren. Dann sieht das Ganze so aus:

In [13]:
df = df[df['AGS'].apply(lambda x: len(x)>3)]
df

,Jahr,AGS,Ort,Art,Einheit,Insgesamt,baureifes Land
307,2018,05111,"Düsseldorf, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,959.98,959.98
311,2018,05112,"Duisburg, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,254.82,277.14
315,2018,05113,"Essen, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,274.93,322.64
319,2018,05114,"Krefeld, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,52.63,146.35
323,2018,05116,"Mönchengladbach, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,85.40,128.48
327,2018,05117,"Mülheim an der Ruhr, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,239.44,239.44
331,2018,05119,"Oberhausen, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,214.06,236.46
335,2018,05120,"Remscheid, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,117.16,125.64
339,2018,05122,"Solingen, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,228.69,228.69
343,2018,05124,"Wuppertal, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,132.95,160.85


Das sieht schon ganz gut aus.
Aber ist dir Aachen aufgefallen?

In [14]:
df[df.Ort.str.contains('Aachen')]

,Jahr,AGS,Ort,Art,Einheit,Insgesamt,baureifes Land
383,2018,05334,Städteregion Aachen (einschl. Stadt Aachen),Durchschnittlicher Kaufwert je qm,EUR,109.75,107.07
387,2018,05334002,"Aachen, krfr. Stadt",Durchschnittlicher Kaufwert je qm,EUR,158.97,157.82
391,2018,05354,"Aachen, Kreis",Durchschnittlicher Kaufwert je qm,EUR,-,-


Aus Gründen wird Aachen in der Regionalstatistik oft mehrfach angegeben.
Wir filtern die kreisfreie Stadt und den Kreis Aachen raus.

In [15]:
df = df[(df.AGS != '05354') & (df.AGS != '05334002') ]

In [16]:
df

,Jahr,AGS,Ort,Art,Einheit,Insgesamt,baureifes Land
307,2018,05111,"Düsseldorf, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,959.98,959.98
311,2018,05112,"Duisburg, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,254.82,277.14
315,2018,05113,"Essen, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,274.93,322.64
319,2018,05114,"Krefeld, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,52.63,146.35
323,2018,05116,"Mönchengladbach, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,85.40,128.48
327,2018,05117,"Mülheim an der Ruhr, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,239.44,239.44
331,2018,05119,"Oberhausen, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,214.06,236.46
335,2018,05120,"Remscheid, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,117.16,125.64
339,2018,05122,"Solingen, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,228.69,228.69
343,2018,05124,"Wuppertal, Kreisfreie Stadt",Durchschnittlicher Kaufwert je qm,EUR,132.95,160.85


## 6. Spalten auswählen

Wir empfehlen, nur die wirklich notwendigen Spalten in Datwrapper zu kopieren. Deshalb nicht nötige Spalten weglassen:

In [17]:
df.columns

Index(['Jahr', 'AGS', 'Ort', 'Art', 'Einheit', 'Insgesamt', 'baureifes Land'], dtype='object')

In [18]:
df = df[['AGS', 'Ort', 'baureifes Land']]

In [19]:
df.head()

,AGS,Ort,baureifes Land
307,05111,"Düsseldorf, Kreisfreie Stadt",959.98
311,05112,"Duisburg, Kreisfreie Stadt",277.14
315,05113,"Essen, Kreisfreie Stadt",322.64
319,05114,"Krefeld, Kreisfreie Stadt",146.35
323,05116,"Mönchengladbach, Kreisfreie Stadt",128.48


## 7. Daten in die Zwischenablage kopieren

Statt die Daten als .csv abzuspeichern, kannst du sie auch in die Zwischenablage kopieren (wie mit STRG-C) und in Datawrapper per STRG-V einfügen. 

In [21]:
df.to_clipboard()

### 8. Baue deine Karte
- In Datawrapper wählst du NRW Landkreise als neue Karte aus
- fügst dann die Daten ein und wählst die entsprechenden Spalten aus (AGS und Wert). AGS steht für 'amtlicher Gemeindeschlüssel'. Datawrapper nutzt diese Gemeindekennziffer, um Daten und Gebiete eindeutig zuzuordnen.
- Danach kannst du die Karte beschriften und deine Datenquelle angeben und die Karte mit Tooltips, Legende etc. nach deinen Wünschen gestalten.
- Am Schluss auf 'Veröffentlichen' klicken.

Prima!

Auf zum nächsten Datenformat:
    
**[>> Json-Notebook](05_json.ipynb)**